In [ ]:
tree_canopy_gdf = gpd.read_file('Trees__Bureau_of_Street_Services_.shp')

In [ ]:
tree_canopy_gdf = tree_canopy_gdf.to_crs(neighborhoods_gdf.crs)

In [ ]:
westlake_bus_stops_with_shelter = westlake_stops[westlake_stops['Shelter'] == 1]
westlake_bus_stops_without_shelter = westlake_stops[westlake_stops['Shelter'] == 0]

In [ ]:
def create_neighborhood_map(neighborhood, bus_stops, neighborhood_name):
    projected_neighborhood = neighborhood.to_crs(epsg=3857)
    
    centroid_x = projected_neighborhood.geometry.centroid.x.mean()
    centroid_y = projected_neighborhood.geometry.centroid.y.mean()
    
    centroid = gpd.GeoSeries([Point(centroid_x, centroid_y)], crs='EPSG:3857').to_crs(epsg=4326)
    
    m = folium.Map(location=[centroid.geometry.y.iloc[0], centroid.geometry.x.iloc[0]], zoom_start=13.3, tiles='OpenStreetMap')
   
    folium.GeoJson(neighborhood.geometry.to_crs(epsg=4326), name=neighborhood_name).add_to(m)

    m = folium.Map(location=[centroid.geometry.y.iloc[0], centroid.geometry.x.iloc[0]], 
                   zoom_start=14, 
                   tiles='CartoDB positron')
    
    folium.GeoJson(projected_neighborhood.geometry.to_crs(epsg=4326), 
               name=neighborhood_name,
               style_function=lambda x: {'fillColor': 'white', 'color': 'black', 'weight': 0.98}).add_to(m)
        # Adding markers to the map
    for _, row in bus_stops.iterrows():
        location = [row.geometry.y, row.geometry.x]
        # If the bus stop has a shelter
        if row['Shelter'] == 1:
            folium.CircleMarker(
                location=location,
                radius=9,  # for the outer circle
                color='purple',  # purple border for outer circle
                fill=True,
                fill_color='purple',  # purple fill for outer circle
                fill_opacity=0.3
            ).add_to(m)
            folium.CircleMarker(
                location=location,
                radius=3,  # for the inner circle
                color='white',  # white border for inner circle
                fill=True,
                fill_color='white',  # white fill for inner circle
                fill_opacity=0.8
            ).add_to(m)
        else:
            # If the bus stop does not have a shelter
            folium.CircleMarker(
                location=location,
                radius=2,
                color='darkgrey',  # dark grey border for circle
                fill=True,
                fill_color='darkgrey',  # dark grey fill for circle
                fill_opacity=0.3
            ).add_to(m)
   
    # Add tree canopy layer to map
            folium.GeoJson(
                tree_canopy.geometry.to_crs(epsg=4326),
                name='Tree Canopy',
                style_function=lambda x: {'fillColor': 'green', 'weight': 0, 'fillOpacity': 0.5}
            ).add_to(m)
    header_html = """
    <div style="position: fixed; 
                top: 10px; 
                left: 50px; 
                width: 200px; 
                z-index:9999; 
                font-size:24px; 
                background-color: rgba(255, 255, 255, 0.8); 
                padding: 10px; 
                border-radius: 7px;">
    <h2 style="margin:0; padding:0;">Silver Lake</h2>
    </div>
    """
    m.get_root().html.add_child(folium.Element(header_html))


    legend_html = """
    <div style="position: fixed; 
            bottom: 50px; 
            left: 50px; 
            width: 180px; 
            height: auto; 
            background-color: white; 
            border-radius: 5px;
            padding: 10px;
            box-shadow: 0 0 11px rgba(0, 0, 0, 0.2);
            z-index:9999;">
    <b>Bus Stop Legend</b> <br>
    <div style="display: flex; align-items: center; margin-bottom: 4px;">
        <div style='width: 18px; height: 18px; border: 5px solid purple; border-radius: 50%; position: relative;'>
            <div style='width: 8px; height: 8px; background: white; border-radius: 50%; position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%);'></div>
        </div>
        <span style="margin-left: 5px;">With Shelter</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 4px;">
        <div style='width: 18px; height: 18px; background: darkgrey; border-radius: 50%;'></div>
        <span style="margin-left: 5px;">Without Shelter</span>
    </div>
    <div style="margin-top: 8px;">
        Total with shelter: 7 <br>
        Total without shelter: 34
    </div>
</div>
"""
    folium.LayerControl().add_to(m)
    
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

silverlake_map = create_neighborhood_map(silverlake_neighborhood, silverlake_bus_stops_with_shelter, silverlake_bus_stops_without_shelter, tree_canopy_gdf, 'Westlake')

# To display the map for Westlake
westlake_map




In [ ]:
def create_neighborhood_map(neighborhood, bus_stops_with_shelter, bus_stops_without_shelter, tree_canopy, neighborhood_name):
    # Project neighborhood to web mercator for folium (EPSG:3857)
    projected_neighborhood = neighborhood.to_crs(epsg=3857)
    
    # Calculate the centroid of the neighborhood for the map center
    centroid_x = projected_neighborhood.geometry.centroid.x.mean()
    centroid_y = projected_neighborhood.geometry.centroid.y.mean()
    
    # Convert the centroid to latitude/longitude for folium (EPSG:4326)
    centroid = gpd.GeoSeries([Point(centroid_x, centroid_y)], crs='EPSG:3857').to_crs(epsg=4326)
    
    # Create the folium map centered on the neighborhood
    m = folium.Map(location=[centroid.geometry.y.iloc[0], centroid.geometry.x.iloc[0]], zoom_start=15.2, tiles='OpenStreetMap')
   
    # Add neighborhood polygon to map
    folium.GeoJson(neighborhood.geometry.to_crs(epsg=4326), name=neighborhood_name).add_to(m)
  
    # Add bus stops with shelters
    for _, row in bus_stops_with_shelter.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color='green',
            fill=True,
            fill_color='green',
            popup='Shelter: Yes'
        ).add_to(m)
    
    # Add bus stops without shelters
    for _, row in bus_stops_without_shelter.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            popup='Shelter: No'
        ).add_to(m)
    
    # Add tree canopy layer to map
    folium.GeoJson(
        tree_canopy.geometry.to_crs(epsg=4326),
        name='Tree Canopy',
        style_function=lambda x: {'fillColor': 'green', 'weight': 0, 'fillOpacity': 0.5}
    ).add_to(m)
    
    # Rest of your code for header_html and legend_html...
    # Add them to map
    m.get_root().html.add_child(folium.Element(header_html))
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add a layer control panel to the map
    folium.LayerControl().add_to(m)
    
    return m
    
westlake_map = create_neighborhood_map(westlake_neighborhood, westlake_bus_stops_with_shelter, westlake_bus_stops_without_shelter, tree_canopy_gdf, 'Westlake')

# To display the map for Westlake
westlake_map


In [ ]:
bus_stops_with_canopy = gpd.sjoin(bus_stops_gdf, tree_canopy_gdf, how="left", predicate='intersects')
bus_stops_with_canopy['has_canopy'] = bus_stops_with_canopy.index_right.notnull()
